In [ ]:
# General Libraries
%matplotlib inline
import pandas as pd
import numpy as np
import os               # para manipulacion de directorios
import shutil           # para eliminar carpetas no vacias

# Oversampling and under sampling
#!pip install -U imbalanced-learn
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from collections import Counter

# Augment
!pip install nlpaug
!pip install transformers
!pip install sacremoses
!pip install sentencepiece
!pip install translate
!pip install --upgrade translatepy
!pip install nltk

from translatepy import Translator
import time
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Cargamos el modelo pre-entrenado y el tokenizador
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')


In [ ]:
# Select the Personality

# 0: Agrado             # Score actual: 0.6830 (svc) | 0.6832 (knn)         | 0.6878(knn & COUNT) |
# 1: Apertura           # Score actual: 0.5916 (svc) | 0.5125 (knn)         | 0.6625 (dt & mf=400 & TF)    | 0.683 (dt & mf=70 & COUNT) | 0.6958 (dt & mf=60 & COUNT & cv=10) | 0.720 (dt & mf=175 & TFIDF & cv=15) | 0.754 (dt & mf=175 & COUNT & cv=15)
# 2: Extraversion       # Score actual: 0.6800 (svc) | 0.6880 (knn)         | 0.7000 (dt & COUNT) |
# 3: Neuroticismo       # Score actual: 0.5819 (svc) | 0.5887 (lr & COUNT)  | 0.6248 (dt & TF)    | 0.6999 (GridSearch Pipeline)
# 4: Responsabilidad    # Score actual: 0.7750 (svc) | 0.7833 (knn & TFIDF) | 0.7875 (dt & TF)    | 0.7958 (dt & TF & not_processed_texts) | 0.8041 (dt & TF & not_processed_texts & cv=10)

POS = 3

personality = ["Agrado", "Apertura", "Extraversion", "Neuroticismo", "Responsabilidad"]
colum_title = ["AgradPerc", "ApertPerc", "ExtravPerc", "NeurPerc", "RespPerc"]
path_colab  = [
    "https://drive.google.com/file/d/138orceXr83_CdJiVvC7bb9IF4-J8P6fp/view", #"./Agrado_Full.csv",
    "https://drive.google.com/file/d/1598rz27p2WRDtq6W1kQF1Pp0uigzpB_U/view", #"./Apertura_Full.csv" ,
    "https://drive.google.com/file/d/187RaPP8EnCjetblpr6YuHuCIXuASVDoa/view", #"./Extraversion_Full.csv" ,

    "https://drive.google.com/file/d/18h2J-jBaPXbhROjLnglukvYaF_Mv4BmN/view", #"./Neuroticismo_Full.csv" ,
    "https://drive.google.com/file/d/1wr20m6Cr7HL91rnbM3t5tfDr5GTZcHpS/view", #"./#Responsabilidad_Full.csv"
]

# Set values
path_colab = path_colab[POS]
colum_labels = colum_title[POS]

# With ONLY URL
if path_colab.startswith('http'):
  print("Ubication WEB")
  _colab='https://drive.google.com/uc?id=' + _colab.split('/')[-2]
else: print("Ubication Local")


Ubication WEB


In [ ]:
# Importar un CSV con pandas a un formato DataFrame (df)

df = pd.read_csv(
    path_colab,
    encoding = "utf-8",
    header = 0,
)

In [ ]:
df[colum_labels].replace("yes", 1, inplace=True)
df[colum_labels].replace("no", 0, inplace=True)

# Sumar los valores de cada columna
pos = sum(df[colum_labels])
neg = len(df[colum_labels])-pos

print(f"Positivos: {pos}")
print(f"Negativos: {neg}")

Positivos: 121
Negativos: 135


In [ ]:
print(f'Personalidad: {colum_labels} \n')

# Para seleccionar NEGATIVA => selection = 0
# Para seleccionar POSITIVA => selection = 1
selection = 1

positive_negative_selection = 'POSITIVA' if selection else 'NEGATIVA'
print(f'Clase { positive_negative_selection } para aplicar Augment')

dataset_text_corpus_pos = df.loc[df[colum_labels] == 1]
dataset_text_corpus_neg = df.loc[df[colum_labels] == 0]

# Aqui selecciono los textos negativos o positivos para realizar aumentación
#dataset_text_corpus = list(dataset_text_corpus_neg["Autodescripción"])
dataset_text_corpus = list(dataset_text_corpus_pos["Autodescripción"]) if selection else list(dataset_text_corpus_neg["Autodescripción"])
print(f'{len(dataset_text_corpus)} textos originalmente')


Personalidad: NeurPerc 

Clase POSITIVA para aplicar Augment
121 textos originalmente


In [ ]:
def split_text(text, numer_of_splits):
  words = text.split()  # dividimos el texto por espacios en blanco
  num_words = len(words)
  chunk_size = num_words // numer_of_splits  # dividimos el número de palabras en 'numer_of_splits' partes iguales
  chunks = [words[i:i+chunk_size] for i in range(0, num_words, chunk_size)]  # creamos una lista con las partes iguales
  array_of_strings = [' '.join(array) for array in chunks]
  return array_of_strings

def gptAugment(prompt, length):
  #prompt = f"Generate a new text from the ideas and content of the following: {text_input}"
  encoded_prompt = tokenizer.encode(prompt, add_special_tokens=False,  return_tensors="pt")   # tokenizamos el prompt
  output = model.generate(encoded_prompt, max_length=length*2, num_return_sequences=1, do_sample=True)    # generamos texto a partir del prompt
  generated_text = tokenizer.decode(output[0], skip_special_tokens=True)    # decodificamos el texto generado
  return generated_text

def backtranslate(text):
  time.sleep(1)  # Esperar 1 segundo antes de traducir nuevamente
  #print("\n- back_translation_before=", text)
  translator = Translator()
  translation = (translator.translate(
      text=text,
      source_language='es',
      destination_language='en'
  )).result
  translation = translation.replace('.', '. ').replace(',', '; ').replace(';', '; ')
  count_words = len(translation.split())

  if (count_words > 500):
    count_words = 500

  translation = gptAugment(translation, count_words)
  translation = translation.split()[count_words:] # Quito las palabras que ingresaron
  translation = ' '.join(translation)
  #time.sleep(2)  # Esperar 1 segundo antes de traducir nuevamente
  back_translation = translator.translate(
      translation,
      source_language='en',
      destination_language='es'
  ).result
  #print("- back_translation_after=", back_translation)
  return back_translation


translation_list = []
print(dataset_text_corpus[0])
augment_dataset = []

to_generate = 1 # Textos a generar por cada uno de los textos de la clase seleccionada

for text in dataset_text_corpus:
  print(f"\n translation_list: {len(augment_dataset)}")
  # Genero {to_generate} textos nuevos a partir del texto original
  text_generated = [] # Aqui alacenaremos los {to_generate} textos nuevos
  for j in range(to_generate):
    # Separo el texto en 4 partes para que el prompt no se sature (cada prompt acepta hasta 500 palabras aprox como maximo)
    splitted_text = split_text(text,4)
    text_concatenated = [] # Aqui vamos a ir concatenando los textos que fueron generados a partir de los 4 fragmentos
    for i in range(4):
      #print(f"fragment {i}: {splitted_text[i]}")
      generated = backtranslate(splitted_text[i]).replace('.', '. ').replace(',', '; ').replace(';', '; ')
      text_concatenated.append(f'. {generated}')

    augment_dataset.append(text_concatenated)
  print(f"augment_dataset: {augment_dataset}")



In [ ]:
augment_dataset_processed = []

# Unir los textos que estaban separados de a dos (tipo1 a tipo2)
# tipo1 = [['hola que tal', ' como estas'], ['bien y',' vos ?']]
# tipo2 = [['hola que tal como estas'], ['bien y vos ?']]

for text in augment_dataset:
    string = " ".join(text)
    string = string.replace('.', '. ').replace(',', '; ').replace(';', '; ')
    augment_dataset_processed.append([string])

print((augment_dataset_processed))

print(len(augment_dataset_processed))

[['.  para entretenimiento;  La forma habitual;  Un comportamiento típico;    una actitud muy común.  Si elijo no dar una razón por la que lo prefiera así;  Lo contrario de cómo me siento;    antes decido no tomar las cosas más.  Siempre elijo cosas que disfruto y no me gustan;  Y tener una actitud muy positiva sobre todo.  Entiendo esto;  Cuando en una habitación.  Si lo hago de nuevo;    siempre pienso en algo negativo;  Si no se sienten cómodos con esta "cosa".  Cuando llega el momento adecuado;    y estoy en el escenario con alguien.  Puse mi mano sobre el hombro de alguien mientras juego un violín y luego hago una expresión triste sobre cómo creo que les gustaría que yo tengan mi presencia.   .  hilo para hilo al hilo al hilo: en el desastre de la vida;    en la otra mitad de un hilo;    un hilo de verdad;    un hilo para estar en el momento.   .  todavía está en Chile en este momento;    así que tuve que decir;  Ok;    ahora tomemos una siesta.  ¿Cuál es el momento a hacer y cuán

In [ ]:
# Manejo de índice (convención para identificar textos generados por Augment)
# Agrado          = +800
# Apertura        = +700
# Extraversion    = +600
# Neuroticismo    = +500
# Responsabilidad = +400

index = 500

for text in augment_dataset_processed:
  nueva_fila = {'ID': index, colum_labels: selection, 'Autodescripción': text}
  #df = df.append(nueva_fila, ignore_index=True) #WARNING statement
  df = pd.concat([df, pd.DataFrame(nueva_fila, index=[0])], ignore_index=True)
  df = df.astype({'ID': 'int'})
  index +=1

print(f'Agregados {len(augment_dataset_processed)} nuevos de tipo {positive_negative_selection} [{selection}]')

Agregados 31 nuevos de tipo POSITIVA [1]


In [ ]:
#Unir df[colum_labels].value_counts()

print(df)
df.to_csv(f'Augment_gpt_{colum_labels}.csv', encoding = "utf-8", index = None, header=True)

      ID  NeurPerc                                    Autodescripción
0      3         0  Mi personalidad no me ayuda demasiado porque s...
1      6         0  En general me considero una persona muy optimi...
2     10         0  En este momento de mi vida me siento una perso...
3     14         0  Soy una persona muy amigable; me gusta mucho c...
4     16         0  Soy una persona muy dedicada a su familia; a s...
..   ...       ...                                                ...
282  526         1  .  para gastarlo;  Mi padre murió el año pasad...
283  527         1  .  con nosotros;  o en mi vida familiar;  Y en...
284  528         1  .  vino;  o como los llamas;  Un tipo diferent...
285  529         1  .  Y siempre quise seguir adelante.  Cómo lleg...
286  530         1  .  Se puede pasar el tiempo ahora mismo intent...

[287 rows x 3 columns]
